<a href="https://colab.research.google.com/github/rezzix/Capstone-Project/blob/master/Toronto_neighbourhood_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmenting and Clustering Neighborhoods in Toronto

### install useful modules

In [10]:
# beautiful soup for web scrapping
!pip install beautifulsoup4
# geocoder for geolocalisation
!pip install geocoder
# folium for map rendering
!pip install folium

### import useful libraries

In [11]:
import numpy as np
import pandas as pd
import requests
import re
import os
from bs4 import BeautifulSoup
import geocoder
from getpass import getpass
import folium

### start scrapping the wikipedia page for neighborhoods of Toronto

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url, allow_redirects=True)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
postalcodes_tab = soup.find('table',class_='wikitable')

neighb_df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
i=0

for neighborhood_tr in postalcodes_tab.find_all('tr'):
  if (len(neighborhood_tr.find_all('td')) == 3) :
    neighb_row = [td.text.rstrip() for td in neighborhood_tr.find_all('td')]
    if (neighb_row[1] != 'Not assigned') :
      neighb_df.loc[i] = neighb_row
      i+=1

neighb_df.head(10)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
neighb_df.shape


(103, 3)

In [6]:
opencage_api_key = getpass("what is your opencage api key : ")
#print (geocoder.opencage('North York, Victoria Village, CA', key=openkage_api_key).latlng)
#print (geocoder.osm('North York, Victoria Village, CA').latlng)

what is your opencage api key : ··········


### add geolocalisation data to the frame

In [7]:
neighb_df['adress'] = neighb_df['PostalCode'] + ', ' + neighb_df['Borough'] +', ' + neighb_df['Neighborhood']+', CA'
#
neighb_df['lat'] = neighb_df['PostalCode']
neighb_df['lng'] = neighb_df['PostalCode']

#neighb_df_tst = neighb_df.head(3)

#neighb_df_tst['coordinates']=neighb_df_tst['adress'].apply(geocoder.osm).apply(lambda x: x.latlng if x != None else None)

for index, row in neighb_df.iterrows():
  latlng = geocoder.opencage(row['adress'], key=opencage_api_key).latlng
  #print (row['adress'], geocoder.opencage(repr(row['adress']), key=opencage_api_key).latlng)
  if (latlng is not None) :
    row['lat'], row['lng'] = latlng[0], latlng[1]

neighb_df
#neighb_df['lat'] = geocoder.osm(neighb_df['adress']).lat
#geocoder.osm('M3A, Parkwoods, North York, CA').latlng

,PostalCode,Borough,Neighborhood,adress,lat,lng
0,M3A,North York,Parkwoods,"M3A, North York, Parkwoods, CA",36.9356,-120.047
1,M4A,North York,Victoria Village,"M4A, North York, Victoria Village, CA",34.5213,-117.397
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","M5A, Downtown Toronto, Regent Park, Harbourfro...",43.6388,-79.383
3,M6A,North York,"Lawrence Manor, Lawrence Heights","M6A, North York, Lawrence Manor, Lawrence Heig...",43.7208,-79.4456
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","M7A, Downtown Toronto, Queen's Park, Ontario P...",34.059,-117.651
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North","M8X, Etobicoke, The Kingsway, Montgomery Road,...",36.1014,-118.848
99,M4Y,Downtown Toronto,Church and Wellesley,"M4Y, Downtown Toronto, Church and Wellesley, CA",43.5501,-80.7164
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...","M7Y, East Toronto, Business reply mail Process...",40.6893,-122.374
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...","M8Y, Etobicoke, Old Mill South, King's Mill Pa...",M8Y,M8Y


In [9]:
address = 'Toronto, CA'

toronto_latlng = geocoder.opencage(address, key=opencage_api_key).latlng

print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_latlng[0], toronto_latlng[1]))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
condition = neighb_df['Borough'].str.contains('Toronto')

neighb_toronto_df = neighb_df[condition]

In [24]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latlng[0], toronto_latlng[1]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighb_toronto_df['lat'], neighb_toronto_df['lng'], neighb_toronto_df['Borough'], neighb_toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto